In [3]:
import numpy as np

import tensorflow_hub as hub
import tensorflow as tf

import re

from naps.pipelines.read_naps import read_naps_dataset
from naps.uast import uast_pprint

from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
#from keras.utils import plot_model

#import elmo
from naps.pipelines.read_naps import read_naps_dataset
#from dataset import flatten, generate_output, get_tokens, even_embeddings, even_tokens
#from tree_transformation import make_the_tree_good
#from loss_function import init_loss, loss_function


import warnings
warnings.filterwarnings("ignore")

C:\ProgramData\Anaconda3\envs\synth\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\envs\synth\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\envs\synth\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\envs\synth\lib\site-packages\tensorflow\python\f

In [6]:
global elmo

def init():
    global elmo
    elmo = hub.Module("https://tfhub.dev/google/elmo/3", trainable=True)


def get_words(text):
    sentences_dirty = text.split('.')

    max_len = 0

    sentences = []
    for sent in sentences_dirty:
        sent = re.sub('[,;-]', '', sent)
        if len(sent) > 0:
            sentences.append(sent)
            splitted = sent.split()
            if len(splitted) > max_len:
                max_len = len(splitted)

    words = []
    mask = []
    masked_words = []
    for sent in sentences:
        splitted = sent.split()
        for i in range(max_len):
            try:
                words.append(splitted[i])
            except:
                words.append('_')

    for word in words:
        if word == "_":
            mask.append(False)
        else:
            mask.append(True)
            masked_words.append(word)
    return sentences, mask

def get_embeddings(text):
    global elmo
    sentences, mask = get_words(text)

    embeddings = elmo(
        sentences,
        signature="default",
        as_dict=True)["elmo"]


    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        sess.run(tf.compat.v1.tables_initializer())
        x = sess.run(embeddings)
    #print(x)
    embs = x.reshape(-1, 1024)
    masked_embs = embs[mask]

    return np.array(masked_embs)

In [7]:
def make_the_tree_good(bad_tree):
    # Zwracana zmienna
    good_tree = []

    # Warunek zakończenia rekurencji
    if type(bad_tree) is not list:
        good_tree = bad_tree

    # Warunek zaistnienia rekurencji
    else:
        for x in bad_tree:
            mttg = make_the_tree_good(x)

            # Dobranie sposobu dołączenia do wynikowej listy
            if type(mttg) is list:
                good_tree.extend(mttg)
            else:
                good_tree.append(str(mttg))

    return good_tree

In [8]:
def flatten(iterable):
    iterator, sentinel, stack = iter(iterable), object(), []
    while True:
        value = next(iterator, sentinel)
        if value is sentinel:
            if not stack:
                break
            iterator = stack.pop()
        elif isinstance(value, str):
            yield str(value)
        else:
            try:
                new_iterator = iter(value)
            except TypeError:
                yield value
            else:
                stack.append(iterator)
                iterator = new_iterator

In [48]:
def get_unique_tokens():
    ds, _, test = read_naps_dataset()
    tokens_total = []
    with ds:
        
        for d in ds:
            if "is_partial" in d and d["is_partial"]:
                continue
            
            tokens = list(set(list(make_the_tree_good(d["code_tree"]["funcs"]))))
            tokens_total += tokens
            
    with test:
        
        for d in test:
            if "is_partial" in d and d["is_partial"]:
                continue
            
            tokens = list(set(list(make_the_tree_good(d["code_tree"]["funcs"]))))
            tokens_total += tokens

    tokens_total = list(set(tokens_total))
    print(tokens_total) 
    print(len(tokens_total)) #1850

    with open('tokens.txt', 'w') as f:
        for token in tokens_total:
            f.write("%s\n" % token)

def get_tokens():
    with open('tokens2.txt', 'r') as f:
        tokens = f.read().splitlines()
        return tokens

def generate_output(tokens, unique):
    values = np.zeros((len(tokens), len(unique)))
    for i in range(len(tokens)):
        index = unique.index(str(tokens[i]))
        values[i][index] = 1
    return values

def generate_tokens(output, unique):
    output = np.array(output)
    tokens = []
    for out in output:
        idx = np.argmax(out)
        tokens.append(unique[idx])
    return tokens

def even_embeddings(embed, n):
    if len(embed) < n:
        diff =  n - len(embed)
        embed = np.vstack([embed, np.zeros((diff,len(embed[0])))])
    return embed

def even_tokens(tokens, n):
    tokens = np.concatenate((['START'],tokens,['END'] ))
    if len(tokens) < n:
        diff =  n - len(tokens)
        tokens = np.concatenate((tokens, np.full(diff, '')))
    return tokens


In [18]:
def prepare_train_data():
    init()
    unique = get_tokens()

    ds, _, test = read_naps_dataset()
    X_train = []
    Y_train = []
    X_test = []
    Y_test = []

    Y_tokens = []
    Y_tokens2 = []
    max_embs_len = 0
    max_tokens_len = 0
    stopper = 100
    with ds:
        for d in ds:
            if "is_partial" in d and d["is_partial"]:
                continue
            masked_embs = get_embeddings(' '.join(d["text"]))
            if len(masked_embs) > max_embs_len:
                max_embs_len = len(masked_embs)
            X_train.append(masked_embs)    
            tokens = make_the_tree_good(d["code_tree"]["funcs"])
            #print(make_the_tree_good(d["code_tree"]["funcs"]))
            if len(tokens) > max_tokens_len:
                max_tokens_len = len(tokens)
            Y_tokens.append(tokens)
            #break

    with test:
        for t in test:
            if "is_partial" in t and t["is_partial"]:
                continue
            masked_embs = get_embeddings(' '.join(t["text"]))
            if len(masked_embs) > max_embs_len:
                max_embs_len = len(masked_embs)
            X_test.append(masked_embs)    
            tokens = make_the_tree_good(t["code_tree"]["funcs"])
            #print(make_the_tree_good(d["code_tree"]["funcs"]))
            if len(tokens) > max_tokens_len:
                max_tokens_len = len(tokens)
            Y_tokens2.append(tokens)
            #break

    X_train_2 = []
    #print(X_train)
    for ex in X_train:
        X_train_2.append(even_embeddings(ex, max_embs_len))
    
    for ex in Y_tokens:
        #print(even_tokens(ex,max_tokens_len))
        Y_train.append(generate_output(even_tokens(ex,max_tokens_len), unique))

    X_test_2 = []
    #print(X_train)
    for ex in X_test:
        X_test_2.append(even_embeddings(ex, max_embs_len))
    
    for ex in Y_tokens2:
        #print(even_tokens(ex,max_tokens_len))
        Y_test.append(generate_output(even_tokens(ex,max_tokens_len), unique))

    return array(X_train_2), array(Y_train), array(X_test_2), array(Y_test) 

In [13]:
def train_model():
    init_loss()
    X_train, Y_train, X_test, Y_test = prepare_train_data()

    model = Sequential()
    #Encoder
    model.add(LSTM(100, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(LSTM(50, activation='relu'))
    #Decoder/Generator
    model.add(RepeatVector(Y_train.shape[1]))
    model.add(LSTM(50, activation='relu', return_sequences=True))
    model.add(LSTM(100, activation='relu', return_sequences=True))
    model.add(TimeDistributed(Dense(1764, activation='softmax')))
    model.compile(optimizer='adam', loss='binary_crossentropy')

    # fit model
    model.fit(X_train, Y_train, epochs=10, verbose=0)
    #print(model.get_weights())
    print(model.summary())
    model.save('wizard')

In [19]:
X_train, Y_train, X_test, Y_test = prepare_train_data()

with open('X_train.npy', 'wb') as f:
    np.save(f, X_train)
    
with open('Y_train.npy', 'wb') as f:
    np.save(f, Y_train)
    
with open('X_test.npy', 'wb') as f:
    np.save(f, X_test)
    
with open('Y_test.npy', 'wb') as f:
    np.save(f, Y_test)
        


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


ERROR:tensorflow:Session failed to close after 30 seconds. Continuing after this point may leave your program in an undefined state.


ERROR:tensorflow:Session failed to close after 30 seconds. Continuing after this point may leave your program in an undefined state.


KeyboardInterrupt: 

In [20]:
init()
unique = get_tokens()

ds, _, test = read_naps_dataset()
X_train = []
Y_train = []
X_test = []
Y_test = []

Y_tokens = []
Y_tokens2 = []
max_embs_len = 0
max_tokens_len = 0
stopper = 100
stopper2 = 50
counter = 0
with ds:
    for d in ds:
        if "is_partial" in d and d["is_partial"]:
            continue
        counter += 1
        masked_embs = get_embeddings(' '.join(d["text"]))
        if len(masked_embs) > max_embs_len:
            max_embs_len = len(masked_embs)
        X_train.append(masked_embs)    
        tokens = make_the_tree_good(d["code_tree"]["funcs"])
        #print(make_the_tree_good(d["code_tree"]["funcs"]))
        if len(tokens) > max_tokens_len:
            max_tokens_len = len(tokens)
        Y_tokens.append(tokens)
        #break
        if counter == stopper:
            break
counter = 0
with test:
    for t in test:
        if "is_partial" in t and t["is_partial"]:
            continue
        counter += 1
        masked_embs = get_embeddings(' '.join(t["text"]))
        if len(masked_embs) > max_embs_len:
            max_embs_len = len(masked_embs)
        X_test.append(masked_embs)    
        tokens = make_the_tree_good(t["code_tree"]["funcs"])
        #print(make_the_tree_good(d["code_tree"]["funcs"]))
        if len(tokens) > max_tokens_len:
            max_tokens_len = len(tokens)
        Y_tokens2.append(tokens)
        if counter == stopper2:
            break
        #break




INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


KeyboardInterrupt: 

In [ ]:
X_train_2 = []
#print(X_train)
for ex in X_train:
    X_train_2.append(even_embeddings(ex, max_embs_len))

for ex in Y_tokens:
    #print(even_tokens(ex,max_tokens_len))
    Y_train.append(generate_output(even_tokens(ex,max_tokens_len), unique))

X_test_2 = []
#print(X_train)
for ex in X_test:
    X_test_2.append(even_embeddings(ex, max_embs_len))

for ex in Y_tokens2:
    #print(even_tokens(ex,max_tokens_len))
    Y_test.append(generate_output(even_tokens(ex,max_tokens_len), unique))

In [23]:
get_unique_tokens()

['', '1e-08', 'Washington', '103', '41', '62', '100006', '47744747', 'North', '42', 'if', '13:31', 'Yes\\n', 'ring topology', 'Jefferson', 'April', 'six', '2', 'Just a legend', '..', 'W', 'VV', '$#@', 'hate', 'func7', 'major', '\\n-1 -1', '1000050', '82', 'JUMP ', '0 0 1', '<int|<int|int>>', '2016', '998', '<>v^', 'three', 'woman', 'zabcdefghijklmnopqrstuvwxy', 'No such line.', '3000001', '7184606427', 'Van Buren', '\\n', '-9223372036854775809', '6550', '-129', 'I become the guy.', 'TEAM 1 WINS', '8444', 'B ', '1005', 'Yes', 'IMPOSSIBLE', 'I hate that I love it', '274', '1073741824', '262144', 'floor', 'glaceon', '84641582', 'a8', '44777474', '77447744', 'BigInteger', '4\\n', 'RAGGGGGGGGGGGGGGH', '-1 ', 'automaton', '&&', '100002', 'J', 'Timur', '47774474', '807239576', '2313', '25600', '4000000000.0', 'Marsel', 'bca', 'Grant', 'If you want to go to a cafe with a wonderful girl, you should call: ', '32', '10000000000', '4096', 'array', '30', '1989', 'NO\\n', 'CODEFO', '1010', 'return',

In [25]:
Y_train = np.load('data/Y_test.npy', allow_pickle = True)

In [27]:

Y_train = np.delete(Y_train, [41, 60, 94, 125, 149, 151, 187, 251, 252, 271, 272, 336, 372, 383, 465, 477])

In [40]:
len(Y_train[0])

268

In [29]:
unique = get_tokens()

In [30]:
max = 0
for t in Y_train:
    if len(t) > max:
        max = len(t)

In [31]:
max


699

In [59]:
Y_train_f = []
max = 701
for ex in Y_train:
    #print(even_tokens(ex,max_tokens_len))
    Y_train_f.append(generate_output(even_tokens(np.array(ex),max), unique))

In [56]:
evened = even_tokens(Y_train[0],max)

In [57]:
len(evened)

701

In [55]:
np.sum(generate_output(even_tokens(Y_train[0],max), unique))

701.0

In [63]:
Y_train_f[100].shape

(701, 1850)

In [58]:
Y_train_f[0]

['func',
 'int',
 '__main__',
 'var',
 'int*',
 'var0',
 'var',
 'int',
 'var1',
 'var',
 'int*',
 'var2',
 'var',
 'int',
 'var3',
 'var',
 'int',
 'var4',
 'var',
 'int',
 'var5',
 'var',
 'int',
 'var6',
 'var',
 'int',
 'var7',
 'var',
 'int',
 'var8',
 'assign',
 'int',
 'var',
 'int',
 'var8',
 'val',
 'int',
 0,
 'assign',
 'int',
 'var',
 'int',
 'var1',
 'invoke',
 'int',
 'len',
 'var',
 'int*',
 'var0',
 'assign',
 'int*',
 'var',
 'int*',
 'var2',
 'invoke',
 'int*',
 '_ctor',
 'var',
 'int',
 'var1',
 'assign',
 'int',
 'var',
 'int',
 'var3',
 'val',
 'int',
 0,
 'while',
 'void',
 'invoke',
 'bool',
 '<',
 'var',
 'int',
 'var3',
 'var',
 'int',
 'var1',
 'assign',
 'int',
 'invoke',
 'int',
 'array_index',
 'var',
 'int*',
 'var2',
 'var',
 'int',
 'var3',
 'invoke',
 'int',
 'array_index',
 'var',
 'int*',
 'var0',
 'invoke',
 'int',
 '-',
 'assign',
 'int',
 'var',
 'int',
 'var8',
 'invoke',
 'int',
 '+',
 'var',
 'int',
 'var8',
 'val',
 'int',
 1,
 'val',
 'int',
 

In [36]:
unique

['',
 '1e-08',
 'START',
 'END',
 'Washington',
 '103',
 '41',
 '62',
 '100006',
 '47744747',
 'North',
 '42',
 'if',
 '13:31',
 'Yes\\n',
 'ring topology',
 'Jefferson',
 'April',
 'six',
 '2',
 'Just a legend',
 '..',
 'W',
 'VV',
 '$#@',
 'hate',
 'func7',
 'major',
 '\\n-1 -1',
 '1000050',
 '82',
 'JUMP ',
 '0 0 1',
 '<int|<int|int>>',
 '2016',
 '998',
 '<>v^',
 'three',
 'woman',
 'zabcdefghijklmnopqrstuvwxy',
 'No such line.',
 '3000001',
 '7184606427',
 'Van Buren',
 '\\n',
 '-9223372036854775809',
 '6550',
 '-129',
 'I become the guy.',
 'TEAM 1 WINS',
 '8444',
 'B ',
 '1005',
 'Yes',
 'IMPOSSIBLE',
 'I hate that I love it',
 '274',
 '1073741824',
 '262144',
 'floor',
 'glaceon',
 '84641582',
 'a8',
 '44777474',
 '77447744',
 'BigInteger',
 '4\\n',
 'RAGGGGGGGGGGGGGGH',
 '-1 ',
 'automaton',
 '&&',
 '100002',
 'J',
 'Timur',
 '47774474',
 '807239576',
 '2313',
 '25600',
 '4000000000.0',
 'Marsel',
 'bca',
 'Grant',
 'If you want to go to a cafe with a wonderful girl, you should

In [37]:
np.concatenate(([1,2],[3,4],[5,6]))

array([1, 2, 3, 4, 5, 6])